In [27]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [28]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
#pc.create_index(
#    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
#)


In [29]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Excellent professor! Makes difficult concepts easy to understand.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great lecturer, but the exams are tough.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Good professor, but sometimes goes too fast.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'History',
  'stars': 5,
  'review': 'Passionate about the subject, very engaging lectures.'},
 {'professor': 'Dr. Linda Davis',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Not very approachable, but the material is interesting.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Knowledgeable and helpful, but assignments are challenging.'},
 {'professor': 'Dr. Jennifer Martinez',
  'subject': 'English Literature',
  'stars': 5,
  'review': 'Incredible professor! Makes

In [30]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [31]:
processed_data[0]

{'values': [-0.014921693,
  -0.051669918,
  -0.034912106,
  0.015413049,
  0.005863941,
  -0.013693304,
  -0.024102284,
  0.036437895,
  -0.017016418,
  -0.031782947,
  0.029067561,
  -0.0056958455,
  -0.012497241,
  -0.016977629,
  0.037394743,
  0.011139547,
  -0.019654222,
  0.014469128,
  0.040213574,
  0.05466977,
  0.024761735,
  -0.026274592,
  0.023106642,
  0.035481043,
  -0.052471604,
  -0.04760977,
  0.021024846,
  0.020727446,
  0.007169912,
  0.0053887484,
  0.075823925,
  -0.0041280333,
  -0.007234564,
  -0.053428452,
  -0.028550344,
  0.029067561,
  -0.033024266,
  0.024102284,
  -0.017546566,
  0.023740232,
  0.033851814,
  0.019188728,
  -0.0056635193,
  0.019369753,
  0.048489038,
  0.01731382,
  -0.052575044,
  -0.022356678,
  0.025110856,
  0.049006253,
  -0.04952347,
  0.026003053,
  0.06185908,
  0.051256143,
  -0.061600473,
  0.033438038,
  0.013059714,
  0.05182508,
  -0.01859393,
  -0.06630714,
  0.04161006,
  0.006080525,
  0.01986111,
  -0.009639621,
  -0.034

In [32]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [33]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}